In [1]:
import xarray as xr
import geopandas as gpd
import pandas as pd
import numpy as np
import shapely.geometry
import shapely
import rasterio
import rioxarray
from rasterio.features import shapes
from shapely.geometry import shape
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
xr_final_SI = xr.open_dataset('Output\\xr_SI_Landcover.nc')

df_landuse_suitability_index = xr_final_SI.to_dataframe()
df_landuse_suitability_index.reset_index(inplace=True)
df_landuse_suitability_index = gpd.GeoDataFrame(df_landuse_suitability_index, geometry =gpd.points_from_xy(df_landuse_suitability_index['lon'],df_landuse_suitability_index['lat']))
df_landuse_suitability_index.crs = {'init': 'epsg:4326'}

## Acquire polygon containing boudary of Thailand
world_filepath = gpd.datasets.get_path('naturalearth_lowres')
world = gpd.read_file(world_filepath)
th = world.loc[world['name'] == 'Thailand'] # get Thailand row
boundaries = th['geometry'] # get Thailand geometry
boundaries.crs = {'init': 'epsg:4326'}
boundaries = boundaries.geometry.unary_union

df_landuse_suitability_index['lccs_class'].loc[~df_landuse_suitability_index.within(boundaries)] = 0

print(df_landuse_suitability_index)

In [ ]:
## First, I try to create a bouadary of THialand for acquiring data##
thailandmap = gpd.read_file('Data\\tha_admbnda_adm1_rtsd_20220121\\tha_admbnda_adm1_rtsd_20220121.shp')
thailandmap.crs = {'init': 'epsg:4326'}
thailandmap['center'] = thailandmap['geometry'].centroid
thailandmap = thailandmap.drop(columns = 'geometry')
thailandmap = thailandmap.set_geometry('center')
thailandmap.crs = {'init': 'epsg:4326'}
print(thailandmap)

    Shape_Leng  Shape_Area                   ADM1_EN          ADM1_TH  \
0     2.417227    0.131339                   Bangkok    กรุงเทพมหานคร   
1     1.695100    0.079262              Samut Prakan      สมุทรปราการ   
2     1.251111    0.053238                Nonthaburi          นนทบุรี   
3     1.884945    0.126983              Pathum Thani         ปทุมธานี   
4     3.041716    0.213938  Phra Nakhon Si Ayutthaya  พระนครศรีอยุธยา   
..         ...         ...                       ...              ...   
72    5.192570    0.385879                     Trang             ตรัง   
73    3.257829    0.316268               Phatthalung           พัทลุง   
74    2.820830    0.160825                   Pattani          ปัตตานี   
75    4.839230    0.365448                      Yala             ยะลา   
76    3.456840    0.366648                Narathiwat         นราธิวาส   

   ADM1_PCODE  ADM1_REF  ADM1ALT1EN  ADM1ALT2EN  ADM1ALT1TH  ADM1ALT2TH  \
0        TH10       NaN         NaN         NaN 

c:\Users\EGAT\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\EGAT\AppData\Local\Temp\ipykernel_11296\2495709284.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  thailandmap['center'] = thailandmap['geometry'].centroid
c:\Users\EGAT\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order ch

In [ ]:
df_landuse_suitability_index_2 = gpd.sjoin_nearest(df_landuse_suitability_index,thailandmap,how='left')
print(df_landuse_suitability_index_2.columns)

c:\Users\EGAT\AppData\Local\Programs\Python\Python311\Lib\site-packages\geopandas\array.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


Index(['lat', 'lon', 'lccs_class', 'SI_BGEC', 'SI_Biomass', 'SI_BGWW',
       'SI_MSW', 'SI_IEW', 'geometry', 'index_right', 'Shape_Leng',
       'Shape_Area', 'ADM1_EN', 'ADM1_TH', 'ADM1_PCODE', 'ADM1_REF',
       'ADM1ALT1EN', 'ADM1ALT2EN', 'ADM1ALT1TH', 'ADM1ALT2TH', 'ADM0_EN',
       'ADM0_TH', 'ADM0_PCODE', 'date', 'validOn', 'validTo'],
      dtype='object')


In [ ]:
for i in df_landuse_suitability_index_2.columns:
    if i not in (['lat', 'lon', 'lccs_class','SI_Solar','SI_Wind', 'SI_BGEC', 'SI_Biomass', 'SI_BGWW',
       'SI_MSW', 'SI_IEW', 'ADM1_EN', 'ADM1_TH']):
        df_landuse_suitability_index_2 = df_landuse_suitability_index_2.drop(columns=i)

print(df_landuse_suitability_index_2.columns)

10
Index(['lat', 'lon', 'lccs_class', 'SI_BGEC', 'SI_Biomass', 'SI_BGWW',
       'SI_MSW', 'SI_IEW', 'ADM1_EN', 'ADM1_TH'],
      dtype='object')


In [ ]:
print(len(df_landuse_suitability_index_2))
print(df_landuse_suitability_index_2)

15957810
                lat         lon  lccs_class  SI_BGEC  SI_Biomass  SI_BGWW  \
0          5.615278   97.345833           0        0           0        0   
1          5.615278   97.348611           0        0           0        0   
2          5.615278   97.351389           0        0           0        0   
3          5.615278   97.354167           0        0           0        0   
4          5.615278   97.356944           0        0           0        0   
...             ...         ...         ...      ...         ...      ...   
15957805  20.462500  105.623611           0        0           0        0   
15957806  20.462500  105.626389           0        0           0        0   
15957807  20.462500  105.629167           0        0           0        0   
15957808  20.462500  105.631944           0        0           0        0   
15957809  20.462500  105.634722           0        0           0        0   

          SI_MSW  SI_IEW    ADM1_EN ADM1_TH  
0              0    

In [ ]:
print(len(df_landuse_suitability_index_2['ADM1_TH'].unique()))

77


In [ ]:
lccs_resolution = 300 ## meters

## Agriculture ###
df_landuse_suitability_index_2['A_BGEC'] = 0
df_landuse_suitability_index_2['A_BGEC'].loc[df_landuse_suitability_index_2['lccs_class'].isin([10,11,12,20,30,40])] = 1
df_landuse_suitability_index_2['A_BGEC']= df_landuse_suitability_index_2['A_BGEC'].astype('float64')

## Shrubs and Grassland ## Short vegetation and agriculture ## Natural Reserve
df_landuse_suitability_index_2['A_Biomass'] = 0
df_landuse_suitability_index_2['A_Biomass'].loc[df_landuse_suitability_index_2['lccs_class'].isin([110,130,120,121,122,200,201,202,140,150,151,152,153,10,11,12,20,30,40,180])] = 1
df_landuse_suitability_index_2['A_Biomass']= df_landuse_suitability_index_2['A_BGEC'].astype('float64')

df_landuse_suitability_index_2['A_BGWW'] = 0
df_landuse_suitability_index_2['A_BGWW'].loc[df_landuse_suitability_index_2['lccs_class'].isin([190])] = 1
df_landuse_suitability_index_2['A_BGWW']= df_landuse_suitability_index_2['A_BGEC'].astype('float64')

df_landuse_suitability_index_2['A_MSW'] = 0
df_landuse_suitability_index_2['A_MSW'].loc[df_landuse_suitability_index_2['lccs_class'].isin([190])] = 1
df_landuse_suitability_index_2['A_MSW']= df_landuse_suitability_index_2['A_BGEC'].astype('float64')

df_landuse_suitability_index_2['A_IEW'] = 0
df_landuse_suitability_index_2['A_IEW'].loc[df_landuse_suitability_index_2['lccs_class'].isin([190])] = 1
df_landuse_suitability_index_2['A_IEW']= df_landuse_suitability_index_2['A_BGEC'].astype('float64')

print(df_landuse_suitability_index_2)

feedstock = pd.read_excel('Data\\สรุปข้อมูลพลังงาน -  ฐานข้อมูลพลังงานประเทศไทย.xlsx',sheet_name='สรุปข้อมูลพลังงาน -  ฐานข้อมูลพ')
feedstock = feedstock.loc[feedstock['แหล่งพลังงานหลัก'].isin(['ชีวมวล','ก๊าซชีวภาพ','ขยะ'])]
feedstock.reset_index(inplace=True,drop=True)
print(feedstock)
print(len(feedstock['จังหวัด'].unique()))

list_province = df_landuse_suitability_index_2['ADM1_TH'].unique()
list_fueltype = feedstock['แหล่งพลังงานหลัก'].unique()
print(list_province)
print(list_fueltype)


C:\Users\EGAT\AppData\Local\Temp\ipykernel_11296\3737605198.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_landuse_suitability_index_2['A_BGEC'].loc[df_landuse_suitability_index_2['lccs_class'].isin([10,11,12,20,30,40])] = 1
C:\Users\EGAT\AppData\Local\Temp\ipykernel_11296\3737605198.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_landuse_suitability_index_2['A_Biomass'].loc[df_landuse_suitability_index_2['lccs_class'].isin([110,130,120,121,122,200,201,202,140,150,151,152,153,10,11,12,20,30,40,180])] = 1
C:\Users\EGAT\AppData\Local\Temp\ipykernel_11296\3737605198.py:14: SettingWithCopyWarning: 
A value 

                lat         lon  lccs_class  SI_BGEC  SI_Biomass  SI_BGWW  \
0          5.615278   97.345833           0        0           0        0   
1          5.615278   97.348611           0        0           0        0   
2          5.615278   97.351389           0        0           0        0   
3          5.615278   97.354167           0        0           0        0   
4          5.615278   97.356944           0        0           0        0   
...             ...         ...         ...      ...         ...      ...   
15957805  20.462500  105.623611           0        0           0        0   
15957806  20.462500  105.626389           0        0           0        0   
15957807  20.462500  105.629167           0        0           0        0   
15957808  20.462500  105.631944           0        0           0        0   
15957809  20.462500  105.634722           0        0           0        0   

          SI_MSW  SI_IEW    ADM1_EN ADM1_TH  A_BGEC  A_Biomass  A_BGWW  A_M

In [ ]:
## BGEC ###
for i in list_province:
    totalareaprovince = df_landuse_suitability_index_2['A_BGEC'].loc[(df_landuse_suitability_index_2['A_BGEC'] > 0) &(df_landuse_suitability_index_2['ADM1_TH'] == i)].sum()
    feedstockpergrid = (feedstock['ศักยภาพพลังงานทดแทน (ktoe)'].loc[(feedstock['จังหวัด'] == i) & (feedstock['แหล่งพลังงานหลัก'] == 'ก๊าซชีวภาพ')].values[0]) / totalareaprovince
    df_landuse_suitability_index_2['A_BGEC'].loc[(df_landuse_suitability_index_2['A_BGEC'] > 0) & (df_landuse_suitability_index_2['ADM1_TH'] == i)] = feedstockpergrid
    print(i, totalareaprovince, feedstockpergrid)

print("total bgec = ", df_landuse_suitability_index_2['A_BGEC'].sum())

C:\Users\EGAT\AppData\Local\Temp\ipykernel_11296\3107014047.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_landuse_suitability_index_2['A_BGEC'].loc[(df_landuse_suitability_index_2['A_BGEC'] > 0) & (df_landuse_suitability_index_2['ADM1_TH'] == i)] = feedstockpergrid


ภูเก็ต 4816.0 0.00012250830564784052
สตูล 19551.0 0.0004521507851260805
ยะลา 16429.0 9.251932558281089e-05
นราธิวาส 23691.0 0.00019627706724072435
สงขลา 48867.0 0.00039965620971207564
ปัตตานี 28017.0 0.00015133668843916195
ตรัง 38067.0 0.0006034097774975701
พัทลุง 40072.0 0.00035011978438810143
กระบี่ 46710.0 0.0016574609291372298
นครศรีธรรมราช 65831.0 0.00041469824246935337
พังงา 23298.0 0.000388016138724354
สุราษฎร์ธานี 72244.0 0.0005914678035546204
ตราด 10966.0 0.0005772387379171986
ระนอง 24863.0 0.00044121787394924185
ชุมพร 48382.0 0.0015770327807862429
ประจวบคีรีขันธ์ 33665.0 0.0009434130402495173
ระยอง 35658.0 0.0006096808570306803
ศรีสะเกษ 88292.0 0.00033547773297694014
จันทบุรี 42009.0 0.001386369587469352
เพชรบุรี 32723.0 0.00019680347156434314
ชลบุรี 31464.0 0.001533816425120773
สมุทรสงคราม 13418.0 2.981070204203309e-06
สมุทรปราการ 10247.0 2.2445593832341173e-05
สมุทรสาคร 7199.0 2.3614390887623283e-05
อุบลราชธานี 103736.0 0.0004847883087838359
กาญจนบุรี 40315.0 0.001414113853

In [ ]:
## Biomass ###
for i in list_province:
    totalareaprovince = df_landuse_suitability_index_2['A_Biomass'].loc[(df_landuse_suitability_index_2['A_Biomass'] > 0) &(df_landuse_suitability_index_2['ADM1_TH'] == i)].sum()
    feedstockpergrid = (feedstock['ศักยภาพพลังงานทดแทน (ktoe)'].loc[(feedstock['จังหวัด'] == i) & (feedstock['แหล่งพลังงานหลัก'] == 'ชีวมวล')].values[0]) / totalareaprovince
    df_landuse_suitability_index_2['A_Biomass'].loc[(df_landuse_suitability_index_2['A_Biomass'] > 0) & (df_landuse_suitability_index_2['ADM1_TH'] == i)] = feedstockpergrid
    print(i,(feedstock['ศักยภาพพลังงานทดแทน (ktoe)'].loc[(feedstock['จังหวัด'] == i) & (feedstock['แหล่งพลังงานหลัก'] == 'ชีวมวล')].values[0]), totalareaprovince, feedstockpergrid)

print("total Biomass = ", df_landuse_suitability_index_2['A_Biomass'].sum())

C:\Users\EGAT\AppData\Local\Temp\ipykernel_11296\1998804626.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_landuse_suitability_index_2['A_Biomass'].loc[(df_landuse_suitability_index_2['A_Biomass'] > 0) & (df_landuse_suitability_index_2['ADM1_TH'] == i)] = feedstockpergrid


ภูเก็ต 4816.0 0.0015490033222591362
สตูล 19551.0 0.007383765536289705
ยะลา 16429.0 0.005575506725911498
นราธิวาส 23691.0 0.0057730783841965306
สงขลา 48867.0 0.005437616387337058
ปัตตานี 28017.0 0.002990327301281365
ตรัง 38067.0 0.009559198255707042
พัทลุง 40072.0 0.004758684368137353
กระบี่ 46710.0 0.026663883536715907
นครศรีธรรมราช 65831.0 0.01341313363005271
พังงา 23298.0 0.012634990127907975
สุราษฎร์ธานี 72244.0 0.022167930900836054
ตราด 10966.0 0.007981032281597664
ระนอง 24863.0 0.00703414712625186
ชุมพร 48382.0 0.0227258071183498
ประจวบคีรีขันธ์ 33665.0 0.013819396999851479
ระยอง 35658.0 0.003030736440630433
ศรีสะเกษ 88292.0 0.010298668055996012
จันทบุรี 42009.0 0.0014273131947915923
เพชรบุรี 32723.0 0.005853375301775509
ชลบุรี 31464.0 0.01060863208746504
สมุทรสงคราม 13418.0 0.001679087792517514
สมุทรปราการ 10247.0 0.0011583878208256074
สมุทรสาคร 7199.0 0.00041533546325878595
อุบลราชธานี 103736.0 0.0028447212153929207
กาญจนบุรี 40315.0 0.02371350613915416
ราชบุรี 51280.0 0.0056768

In [ ]:
## A_BGWW ###
for i in list_province:
    totalareaprovince = df_landuse_suitability_index_2['A_BGWW'].loc[(df_landuse_suitability_index_2['A_BGWW'] > 0) &(df_landuse_suitability_index_2['ADM1_TH'] == i)].sum()
    feedstockpergrid = (feedstock['ศักยภาพพลังงานทดแทน (ktoe)'].loc[(feedstock['จังหวัด'] == i) & (feedstock['แหล่งพลังงานหลัก'] == 'ขยะ')].values[0]) / totalareaprovince
    df_landuse_suitability_index_2['A_BGWW'].loc[(df_landuse_suitability_index_2['A_BGWW'] > 0) & (df_landuse_suitability_index_2['ADM1_TH'] == i)] = feedstockpergrid
    # print(i, totalareaprovince, feedstockpergrid)

print("total A_BGWW = ", df_landuse_suitability_index_2['A_BGWW'].sum())

## A_MSW ###
for i in list_province:
    totalareaprovince = df_landuse_suitability_index_2['A_MSW'].loc[(df_landuse_suitability_index_2['A_MSW'] > 0) &(df_landuse_suitability_index_2['ADM1_TH'] == i)].sum()
    feedstockpergrid = (feedstock['ศักยภาพพลังงานทดแทน (ktoe)'].loc[(feedstock['จังหวัด'] == i) & (feedstock['แหล่งพลังงานหลัก'] == 'ขยะ')].values[0]) / totalareaprovince
    df_landuse_suitability_index_2['A_MSW'].loc[(df_landuse_suitability_index_2['A_MSW'] > 0) & (df_landuse_suitability_index_2['ADM1_TH'] == i)] = feedstockpergrid
    # print(i, totalareaprovince, feedstockpergrid)

print("total A_MSW = ", df_landuse_suitability_index_2['A_MSW'].sum())

## A_IEW ###
for i in list_province:
    totalareaprovince = df_landuse_suitability_index_2['A_IEW'].loc[(df_landuse_suitability_index_2['A_IEW'] > 0) &(df_landuse_suitability_index_2['ADM1_TH'] == i)].sum()
    feedstockpergrid = (feedstock['ศักยภาพพลังงานทดแทน (ktoe)'].loc[(feedstock['จังหวัด'] == i) & (feedstock['แหล่งพลังงานหลัก'] == 'ขยะ')].values[0]) / totalareaprovince
    df_landuse_suitability_index_2['A_IEW'].loc[(df_landuse_suitability_index_2['A_IEW'] > 0) & (df_landuse_suitability_index_2['ADM1_TH'] == i)] = feedstockpergrid
    # print(i, totalareaprovince, feedstockpergrid)

print("total A_IEW = ", df_landuse_suitability_index_2['A_IEW'].sum())

C:\Users\EGAT\AppData\Local\Temp\ipykernel_11296\3945860369.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_landuse_suitability_index_2['A_BGWW'].loc[(df_landuse_suitability_index_2['A_BGWW'] > 0) & (df_landuse_suitability_index_2['ADM1_TH'] == i)] = feedstockpergrid


total A_BGWW =  6477.519999999995


C:\Users\EGAT\AppData\Local\Temp\ipykernel_11296\3945860369.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_landuse_suitability_index_2['A_MSW'].loc[(df_landuse_suitability_index_2['A_MSW'] > 0) & (df_landuse_suitability_index_2['ADM1_TH'] == i)] = feedstockpergrid


total A_MSW =  6477.519999999995


C:\Users\EGAT\AppData\Local\Temp\ipykernel_11296\3945860369.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_landuse_suitability_index_2['A_IEW'].loc[(df_landuse_suitability_index_2['A_IEW'] > 0) & (df_landuse_suitability_index_2['ADM1_TH'] == i)] = feedstockpergrid


total A_IEW =  6477.519999999995


In [ ]:
print(df_landuse_suitability_index_2)

                lat         lon  lccs_class  SI_BGEC  SI_Biomass  SI_BGWW  \
0          5.615278   97.345833           0        0           0        0   
1          5.615278   97.348611           0        0           0        0   
2          5.615278   97.351389           0        0           0        0   
3          5.615278   97.354167           0        0           0        0   
4          5.615278   97.356944           0        0           0        0   
...             ...         ...         ...      ...         ...      ...   
15957805  20.462500  105.623611           0        0           0        0   
15957806  20.462500  105.626389           0        0           0        0   
15957807  20.462500  105.629167           0        0           0        0   
15957808  20.462500  105.631944           0        0           0        0   
15957809  20.462500  105.634722           0        0           0        0   

          SI_MSW  SI_IEW    ADM1_EN ADM1_TH  A_BGEC  A_Biomass  A_BGWW  A_M

In [ ]:
df_landuse_suitability_index_2 = df_landuse_suitability_index_2.set_index(['lat', 'lon'])
xr_areafeedstock = xr.Dataset.from_dataframe(df_landuse_suitability_index_2)
print(xr_areafeedstock)

<xarray.Dataset>
Dimensions:     (lat: 5346, lon: 2985)
Coordinates:
  * lat         (lat) float64 5.615 5.618 5.621 5.624 ... 20.46 20.46 20.46
  * lon         (lon) float64 97.35 97.35 97.35 97.35 ... 105.6 105.6 105.6
Data variables: (12/13)
    lccs_class  (lat, lon) uint8 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0
    SI_BGEC     (lat, lon) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0
    SI_Biomass  (lat, lon) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0
    SI_BGWW     (lat, lon) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0
    SI_MSW      (lat, lon) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0
    SI_IEW      (lat, lon) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0
    ...          ...
    ADM1_TH     (lat, lon) object 'ภูเก็ต' 'ภูเก็ต' ... 'บึงกาฬ' 'บึงกาฬ'
    A_BGEC      (lat, lon) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    A_Biomass   (lat, lon) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    A_BGWW      